In [1]:

import os
import numpy as np
import pandas as pd
from copy import deepcopy as copy
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

import subprocess
import pickle
from scipy.stats import describe
from tqdm import tqdm

from astropy.io import fits

from lvmdrp.core import image
from lvmdrp.functions import imageMethod, rssMethod

import seaborn as sns

sns.set(context="talk", style="ticks", palette="husl", color_codes=True)

import warnings
warnings.filterwarnings("ignore")


def read_nist_reference(filename, wl_window):
    wli, wlf = wl_window
    ref_nist_or = pd.read_csv(
        filename, sep="\s+", comment="#").set_index("Wavelength")
    ref_nist_or = ref_nist_or.loc[(
        wli <= ref_nist_or.index) & (ref_nist_or.index <= wlf)]

    del_wl = np.diff(ref_nist_or.index).min()
    new_wl = np.unique(
        np.arange(wli, wlf+del_wl, del_wl).tolist()+ref_nist_or.index.to_list())
    ref_nist_rs = ref_nist_or.reindex(new_wl)

    return ref_nist_or, ref_nist_rs


In [2]:
from lvmdrp.utils.metadata import get_metadata


lvm_frames = get_metadata()

# Arcs (4819 – 5088):
# 4819 – 4863 Darks (10 s, 30 s, 90 s)
# 4864 – 4908 Argon  (10 s, 30 s, 90 s)   
# 4909 – 4953 Xenon  (10 s, 30 s, 90 s)
# 4954 – 4998 Mercury-Argon  (10 s, 30 s, 90 s)
# 4999 – 5043 Neon  (10 s, 30 s, 90 s)
# 5044 – 5088 Mercury-Neon  (10 s, 30 s, 90 s)
# Continuums (4159 – 4278):
# 4159 – 4188 Fiber Flats for Red and Blue channels with LDLS (50 s exposure) with darks. 
# 4189 – 4218 Fiber Flats for Red and Blue channels with LDLS (6 s exposure) with darks. 
# 4219 – 4248 Fiber Flats for NIR channel with LDLS (4 s exposure) with darks.
# 4249 – 4278 Fiber Flats for NIR channel with LDLS (0.1 s exposure) with darks.
targ_idxs = np.arange(4819, 5088+1, 1, dtype=int)
cont_idxs = np.arange(4159, 4278+1, 1, dtype=int)

In [3]:
cont_frames = lvm_frames.loc[lvm_frames.label.str.split("-").apply(lambda r: int(r[-1])).isin(cont_idxs)].sort_values("exptime", ascending=False)
targ_frames = lvm_frames.loc[lvm_frames.label.str.split("-").apply(lambda r: int(r[-1])).isin(targ_idxs)].sort_values("exptime", ascending=False)

cont_frames

,datetime,mjd,spec,ccd,exptime,imagetyp,obstime,observat,label,path,...,ffs,mi150,ts,ldls,nirled,reduction_started,reduction_finished,status,flags,calib_id
id,,,,,,,,,,,,,,,,,,,,,
15318,2022-11-23 07:48:04.600961,59762,sp2,b2,50.0,dark,2022-07-02T04:59:12.233,LCO,sdR-s-b2-00004162,/home/mejia/Research/UNAM/lvm-drp/data.sdss5.o...,...,0,0,0,0,0,None,None,1,1,None
15315,2022-11-23 07:48:04.600797,59762,sp2,z2,50.0,object,2022-07-02T05:34:18.466,LCO,sdR-s-z2-00004183,/home/mejia/Research/UNAM/lvm-drp/data.sdss5.o...,...,0,0,0,0,0,None,None,1,1,None
15356,2022-11-23 07:48:04.603432,59762,sp2,r2,50.0,object,2022-07-02T05:27:36.061,LCO,sdR-s-r2-00004179,/home/mejia/Research/UNAM/lvm-drp/data.sdss5.o...,...,0,0,0,0,0,None,None,1,1,None
15449,2022-11-23 07:48:04.608623,59762,sp2,z2,50.0,object,2022-07-02T04:54:10.729,LCO,sdR-s-z2-00004159,/home/mejia/Research/UNAM/lvm-drp/data.sdss5.o...,...,0,0,0,0,0,None,None,1,1,None
15453,2022-11-23 07:48:04.608856,59762,sp2,b2,50.0,object,2022-07-02T04:54:10.729,LCO,sdR-s-b2-00004159,/home/mejia/Research/UNAM/lvm-drp/data.sdss5.o...,...,0,0,0,0,0,None,None,1,1,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15514,2022-11-23 07:48:04.612531,59762,sp2,r2,0.1,dark,2022-07-02T08:32:22.530,LCO,sdR-s-r2-00004252,/home/mejia/Research/UNAM/lvm-drp/data.sdss5.o...,...,0,0,0,0,0,None,None,1,1,None
15513,2022-11-23 07:48:04.612477,59762,sp2,r2,0.1,dark,2022-07-02T08:49:09.153,LCO,sdR-s-r2-00004272,/home/mejia/Research/UNAM/lvm-drp/data.sdss5.o...,...,0,0,0,0,0,None,None,1,1,None
15074,2022-11-23 07:48:04.586757,59762,sp2,r2,0.1,object,2022-07-02T08:53:19.195,LCO,sdR-s-r2-00004277,/home/mejia/Research/UNAM/lvm-drp/data.sdss5.o...,...,0,0,0,0,0,None,None,1,1,None


In [4]:
fiberflat_path = "_data-lvm/fiberflats-spec2"
if not os.path.isdir(fiberflat_path):
    os.makedirs(fiberflat_path, exist_ok=True)

ccds = ["b1", "r1", "z1"]
for idx, frame in tqdm(cont_frames.iterrows(), total=len(cont_frames), desc="processing continuum frames", ascii=True, unit="frame"):

    cont_path = frame.path
    cont_name = frame.label

    hdr = fits.getheader(cont_path)
    if hdr["IMAGETYP"] != "object": continue

    if os.path.isfile(f"{fiberflat_path}/{cont_name}.ms.fits"): continue

    imageMethod.preprocRawFrame_drp(
        in_image=cont_path,
        out_image=f"{fiberflat_path}/{cont_name}.pre.fits",
        boundary_x="1,2040",
        boundary_y="1,2040",
        positions="00,10,01,11",
        orientation="S,S,S,S"
    )

    imageMethod.LACosmic_drp(
        image=f"{fiberflat_path}/{cont_name}.pre.fits",
        out_image=f"{fiberflat_path}/{cont_name}.cosmic.fits",
        increase_radius=1, flim="1.3", parallel='5'
    )

    imageMethod.findPeaksAuto_drp(
        image=f"{fiberflat_path}/{cont_name}.pre.fits",
        out_peaks_file=f"{fiberflat_path}/{cont_name}.trace.peaks",
        disp_axis="X", threshold="5000", slice="3696", nfibers="41", median_box="1", median_cross="1", method="hyperbolic", init_sigma="0.5", verbose=0
    )

    imageMethod.tracePeaks_drp(
        image=f"{fiberflat_path}/{cont_name}.pre.fits",
        peaks_file=f"{fiberflat_path}/{cont_name}.trace.peaks", trace_out=f"{fiberflat_path}/{cont_name}.trc.fits",
        steps=30, method="hyperbolic", threshold_peak=50, poly_disp=5, coadd=30, verbose=0
    )

    imageMethod.subtractStraylight_drp(
        image=f"{fiberflat_path}/{cont_name}.pre.fits",
        trace=f"{fiberflat_path}/{cont_name}.trc.fits",
        stray_image=f"{fiberflat_path}/{cont_name}.back.fits",
        clean_image=f"{fiberflat_path}/{cont_name}.stray.fits",
        aperture=40, poly_cross=2, smooth_gauss=30
    )

    imageMethod.traceFWHM_drp(
        image=f"{fiberflat_path}/{cont_name}.stray.fits",
        trace=f"{fiberflat_path}/{cont_name}.trc.fits",
        fwhm_out=f"{fiberflat_path}/{cont_name}.fwhm.fits",
        blocks=32, steps=30, coadd=20, threshold_flux=50.0, poly_disp=5, clip="1.5,4.0"
    )

    imageMethod.extractSpec_drp(
        image=f"{fiberflat_path}/{cont_name}.stray.fits",
        trace=f"{fiberflat_path}/{cont_name}.trc.fits",
        out_rss=f"{fiberflat_path}/{cont_name}.ms.fits",
        fwhm=f"{fiberflat_path}/{cont_name}.fwhm.fits",
        parallel=5, method="optimal"
    )

processing continuum frames:  25%|##5       | 91/360 [14:49:55<44:42:39, 598.36s/frame] 